In [10]:
!pip install requests pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [61]:
import requests


url = "https://api.crunchbase.com/v4/data/searches/organizations"
api_key = '0c432e863ca5706ec8171730c3f87d36'

headers = {
    'X-cb-user-key': '0c432e863ca5706ec8171730c3f87d36',
    'accept': 'application/json',
    'content-type': 'application/json'
}

data = {
    "field_ids": ["identifier"],
    "limit": 1000,
    "order": [{
        "field_id": "rank_org",
        "nulls": "first",
        "sort": "asc"
    }],
    "query": [{
        "field_id": "location_identifiers",
        "operator_id": "includes",
        "type": "predicate",
        "values": ["83ead471-332b-d02e-67b7-67279aed075b"]
    }]
}

response = requests.post(url, headers=headers, json=data)

if response.status_code == 200:
    entity_list = []
    hrefs =[]
    entity_list = response.json().get('entities')
    for entity in entity_list:
        hrefs.append(entity.get('properties').get('identifier').get('permalink'))
    


In [62]:
def save_list_to_file(lst, filename):
    with open(filename, 'w') as file:
        for item in lst:
            file.write("%s\n" % item)


save_list_to_file(hrefs, 'startups.txt')


In [64]:
import os
import requests
import time


if os.path.exists('startups.txt'):
    startups = []
    with open('startups.txt', 'r') as f:
        for line in f:
            startups.append(line.strip())
    
    hrefs = []
    links = []
    for startup in startups:
        url = 'https://api.crunchbase.com/v4/data/autocompletes?query=' + startup
        while True:
            response = requests.get(url=url, headers={'X-cb-user-key': api_key})
            if response.status_code == 200:
                try:
                    href = response.json().get("entities")[0].get("identifier").get("permalink")
                except IndexError as e:
                    print(e)
                hrefs.append(href)
                print(href)
                break
            elif response.status_code == 429:
                print('Rate limited. Waiting before retrying...')
                time.sleep(60)
            else:
                print('Endpoint returned:', response.status_code)
                break
else:
    print("startups.txt file not found")


mikat
ag-biotech
mpact-collective
tutorial-systems
jimeta-systems
bentley-records-llc
ez-dental-billing
brightfluence
quick-access-capital
sustainable-business-hub
rolling-steel-industries
wpgroupbuy
trading-physics
khan-construction-development
satya-systems
referazon
popcorn-time
gr0wth-dr1vers
mac-security
sino-sci
crosby-street-ventures
prediction-marketing
jw-brower-associates
bluebird-music
corporate-comm
robert-v-moriarty-md
compliance-and-capacity-skills-international
Rate limited. Waiting before retrying...
fisch-financial-group
sellerscourt
converge-marketing-services
avcas
xomox-jewelry
susie-saltzman
crosswalk-ventures
molly-cagwin
nofo-real-estate
anchor-veterinary-surgery
dark-horse-advisors
ed-ryan-real-estate-group
north-point-partners
knox-design-strategy
playsecure
queens-family-dental
seracle
dr-tros-medical-weight-loss-direct-primary-care
sqrblock-solutions
early-riders
zufall-communications
electronx
concord-private-jet
b-brand-agency
custom-syndicate
raiseway
joan

In [65]:
import pandas as pd
import requests
import time


data_list = []

for href in hrefs:
    url = "https://api.crunchbase.com/api/v4/entities/organizations/" + href
    
    params = {
        "field_ids": "identifier,image_url,linkedin,website_url,short_description,location_identifiers",
        "user_key": api_key
    }
    
    while True:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            properties = data.get('properties', {})
            name = properties.get('identifier', {}).get('value', '')
            image_url = properties.get('image_url', '')
            description = properties.get('short_description', '')
            website = properties.get('website_url', '')
            linkedin = properties.get('linkedin', {}).get('value', '')
            locations = properties.get('location_identifiers', [])
            geo_location = ", ".join(location.get('value', '') for location in locations)
            cb_link = "https://www.crunchbase.com/organization/"
            cb_url = cb_link + href
            data_list.append({
                'name': name,
                'image_url': image_url,
                'description': description,
                'website': website,
                'linkedin': linkedin,
                'crunchbase_url': cb_url,
                'headquarters_location': geo_location,
            })
            print(href)
            break
        elif response.status_code == 429:
            print('Rate limited. Waiting before retrying...')
            time.sleep(60) 
        else:
            print(f"Failed to retrieve data for {href}: {response.status_code}")
            break

df = pd.DataFrame(data_list)
df.to_csv('startup_data.csv')
print('Successfully extracted details to startup_data.csv')


mikat
ag-biotech
mpact-collective
tutorial-systems
jimeta-systems
bentley-records-llc
ez-dental-billing
brightfluence
quick-access-capital
sustainable-business-hub
rolling-steel-industries
wpgroupbuy
trading-physics
khan-construction-development
satya-systems
referazon
popcorn-time
gr0wth-dr1vers
mac-security
sino-sci
crosby-street-ventures
prediction-marketing
jw-brower-associates
bluebird-music
corporate-comm
robert-v-moriarty-md
Rate limited. Waiting before retrying...
compliance-and-capacity-skills-international
fisch-financial-group
sellerscourt
converge-marketing-services
avcas
xomox-jewelry
susie-saltzman
crosswalk-ventures
molly-cagwin
nofo-real-estate
anchor-veterinary-surgery
dark-horse-advisors
ed-ryan-real-estate-group
north-point-partners
knox-design-strategy
playsecure
queens-family-dental
seracle
dr-tros-medical-weight-loss-direct-primary-care
sqrblock-solutions
early-riders
zufall-communications
electronx
concord-private-jet
b-brand-agency
custom-syndicate
Rate limited.